In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from skimage.measure import shannon_entropy

data_dir = "data"
path_good = os.path.join(data_dir, "good")
path_bad = os.path.join(data_dir, "bad")
resize_size = (128, 128)

def process_image(image_path, resize_size):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, resize_size, interpolation=cv2.INTER_AREA)
    img_equalized = cv2.equalizeHist(img_resized)
    img_blurred = cv2.GaussianBlur(img_equalized, (5, 5), 0)
    return img_blurred

def get_features(image):
    sharpness = cv2.Laplacian(image, cv2.CV_64F).var()
    edges = cv2.Canny(image, 100, 200)
    edge_density = edges.sum() / (resize_size[0] * resize_size[1])
    entropy = shannon_entropy(image)
    return np.array([sharpness, edge_density, entropy])

def load_images(path_good, path_bad, resize_size):
    features, labels = [], []
    for file_name in os.listdir(path_good):
        file_path = os.path.join(path_good, file_name)
        if not file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue
        img = process_image(file_path, resize_size)
        features.append(get_features(img))
        labels.append(1)
    for file_name in os.listdir(path_bad):
        file_path = os.path.join(path_bad, file_name)
        if not file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue
        img = process_image(file_path, resize_size)
        features.append(get_features(img))
        labels.append(0)
    return np.array(features), np.array(labels)

features, labels = load_images(path_good, path_bad, resize_size)
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, random_state=42)
use_rf = True

if use_rf:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
else:
    model = SVC(kernel='rbf', C=1.0, random_state=42)

model.fit(train_features, train_labels)
predictions = model.predict(test_features)

# Calculate metrics
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions, pos_label=1)
recall = recall_score(test_labels, predictions, pos_label=1)

# Print results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")


Accuracy: 0.87
Precision: 0.90
Recall: 0.95
